# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
#Read data from CSV file
weather_df = pd.read_csv("weather_city.csv",index_col=0)
#Print head to verify
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,100,PF,2020-10-25,82,-23.12,-134.97,23.61,10.08
1,Mar del Plata,90,AR,2020-10-25,100,-38.00,-57.56,16.11,1.36
2,Brigantine,90,US,2020-10-25,93,39.41,-74.36,12.78,3.60
3,Tuatapere,96,NZ,2020-10-25,57,-46.13,167.68,12.22,3.13
4,Katsuura,75,JP,2020-10-25,62,35.13,140.30,17.22,3.10
...,...,...,...,...,...,...,...,...,...
570,Santiago del Estero,46,AR,2020-10-25,67,-27.80,-64.26,25.24,11.33
571,Cao Bang,51,VN,2020-10-25,77,22.67,106.25,20.89,0.51
572,San Benito,45,US,2020-10-25,57,36.62,-121.08,20.56,1.34
573,Yatou,64,CN,2020-10-25,67,37.15,122.38,14.37,1.03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Set api key
gmaps.configure(api_key=g_key)
#Set lat and lng as locations
locations = weather_df[["Lat","Lng"]]
#Set Humidity as weight
weight=weather_df["Humidity"]


In [11]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(weight)

# Create heat layer
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights = weight, 
    dissipating=False, 
    max_intensity=100, 
    point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map